In [2]:
!pip install tika
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32625 sha256=da3eca0e29b5230daf273496a8c011fccde90fdb67d49d2f5d117ed117000bad
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [15]:
#import regular expressions
import re 
from tika import parser
import spacy
from spacy.matcher import Matcher
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [32]:
file = r'Pratham Shankwalker - Resume.pdf'
file_data = parser.from_file(file)
text = file_data['content']
print(type(text))

<class 'str'>


In [19]:
# declare a dictionary to store all the extracted text
extracted_text = {}

In [34]:
#function to extract email adress
def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)
  
email = get_email_addresses(text)
print(email)

extracted_text["E-Mail"] = email[0]
extracted_text

['pshankwalker1@gmail.com', 'pshankwalker1@gmail.com']


{'E-Mail': 'pshankwalker1@gmail.com', 'Phone Number': ['7839453651']}

In [36]:
#function to get phone number
def get_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

phone_number= get_phone_numbers(text)
print(phone_number)

#add phone number to the extracted text 
extracted_text["Phone Number"] = phone_number

['9307051204']


In [37]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

name = extract_name(text)
print(name)


Github |


In [51]:
def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    skills = ["machine learning",
             "deep learning",
             "nlp",
             "natural language processing",
             "mysql",
             "sql",
             "django",
             "computer vision",
              "tensorflow",
             "opencv",
             "mongodb",
             "artificial intelligence",
             "ai",
             "flask",
             "robotics",
             "data structures",
             "python",
             "c++",
             "matlab",
             "css",
             "html",
             "github",
             "php",
             "Neural Networks",
              "statistics",
              "Transfer Learning"]
            
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


skills = []
skills = extract_skills(text)
extracted_text['skills']=skills
skills

['Github',
 'Machine learning',
 'Opencv',
 'Natural language processing',
 'Tensorflow',
 'Data structures',
 'Python',
 'Mysql',
 'C++',
 'Sql']

In [40]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'B TECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

education = extract_education(text)
education
extracted_text["Qualification"] = education

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
extracted_text

{'E-Mail': 'pshankwalker1@gmail.com',
 'Phone Number': ['9307051204'],
 'Qualification': [('BE', '2024')],
 'Institutes': ['Goa College of Engineering'],
 'Institute': ['Goa College of Engineering'],
 'Experience': [],
 'skills': ['Github',
  'Machine learning',
  'Opencv',
  'Natural language processing',
  'Tensorflow',
  'Data structures',
  'Python',
  'Mysql',
  'C++',
  'Sql']}

In [46]:
#college/institution
sub_patterns = ['[A-Z][a-z]* College of Engineering','[A-Z][a-z]* Educational Institute',
                'University of [A-Z][a-z]*',
                'Ecole [A-Z][a-z]*']
pattern = '({})'.format('|'.join(sub_patterns))
matches = re.findall(pattern, text)

extracted_text["Institute"] = matches
matches

['Goa College of Engineering']

In [50]:
#experience

sub_patterns = ['[A-Z][a-z]* [A-Z][a-z]* Private Limited','[A-Z][a-z]* [A-Z][a-z]* Pvt. Ltd.','[A-Z][a-z]* [A-Z][a-z]* Inc.', '[A-Z][a-z]* LLC',
                ]
pattern = '({})'.format('|'.join(sub_patterns))
Exp = re.findall(pattern, text)
extracted_text["Experience"] = Exp
Exp

[]